<div style="text-align: center;">
    <h1 style="color: #FF6347;">Self-Guided Lab: Retrieval-Augmented Generation (RAGs)</h1>
</div>

<div style="text-align: center;">
    <img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExZ3FsdzRveTBrenMxM3VnbDMwaTJxN2NnZm50aGFibXk1NzNnY2Q0MCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/LR5ZBwZHv02lmpVoEU/giphy.gif" alt="NLP Gif" style="width: 300px; height: 150px; object-fit: cover; object-position: center;">
</div>

<h1 style="color: #FF6347;">Data Storage & Retrieval</h1>


<h2 style="color: #FF8C00;">PyPDFLoader</h2>

`PyPDFLoader` is a lightweight Python library designed to streamline the process of loading and parsing PDF documents for text processing tasks. It is particularly useful in Retrieval-Augmented Generation workflows where text extraction from PDFs is required.

- **What Does PyPDFLoader Do?**
  - Extracts text from PDF files, retaining formatting and layout.
  - Simplifies the preprocessing of document-based datasets.
  - Supports efficient and scalable loading of large PDF collections.

- **Key Features:**
  - Compatible with popular NLP libraries and frameworks.
  - Handles multi-page PDFs and embedded images (e.g., OCR-compatible setups).
  - Provides flexible configurations for structured text extraction.

- **Use Cases:**
  - Preparing PDF documents for retrieval-based systems in RAGs.
  - Automating the text extraction pipeline for document analysis.
  - Creating datasets from academic papers, technical manuals, and reports.


In [6]:
%pip install langchain langchain_community pypdf
%pip install termcolor langchain_openai langchain-huggingface sentence-transformers chromadb langchain_chroma tiktoken openai python-dotenv


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
import warnings
warnings.filterwarnings('ignore')

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
import os
import warnings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

<h3 style="color: #FF8C00;">Loading the Documents</h3>

In [12]:
# File path for the document

file_path = r"C:\Users\nahia\OneDrive\Escritorio\Ironhack_Bootcamp\Woche_Sieben\lab-intro-rag\ai-for-everyone.pdf"

<h3 style="color: #FF8C00;">Documents into pages</h3>

The `PyPDFLoader` library allows efficient loading and splitting of PDF documents into smaller, manageable parts for NLP tasks.

This functionality is particularly useful in workflows requiring granular text processing, such as Retrieval-Augmented Generation (RAG).


In [13]:
# Load and split the document
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
len(pages)

297

<h3 style="color: #FF8C00;">Pages into Chunks</h3>


####  RecursiveCharacterTextSplitter in LangChain

The `RecursiveCharacterTextSplitter` is the **recommended splitter** in LangChain when you want to break down long documents into smaller, semantically meaningful chunks — especially useful in **RAG pipelines**, where clean context chunks lead to better LLM responses.

####  Parameters

| Parameter       | Description                                                                 |
|-----------------|-----------------------------------------------------------------------------|
| `chunk_size`    | The **maximum number of characters** allowed in a chunk (e.g., `1000`).     |
| `chunk_overlap` | The number of **overlapping characters** between consecutive chunks (e.g., `200`). This helps preserve context continuity. |

####  How it works
`RecursiveCharacterTextSplitter` attempts to split the text **intelligently**, trying the following separators in order:
1. Paragraphs (`"\n\n"`)
2. Lines (`"\n"`)
3. Sentences or words (`" "`)
4. Individual characters (as a last resort)

This makes it ideal for handling **natural language documents**, such as PDFs, articles, or long reports, without breaking sentences or paragraphs in awkward ways.



In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(pages)

len(chunks)

1096

####  Alternative: CharacterTextSplitter

`CharacterTextSplitter` is a simpler splitter that breaks text into chunks based **purely on character count**, without trying to preserve any natural language structure.

##### Example:
```python
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
````

This method is faster and more predictable but may split text in the middle of a sentence or paragraph, which can hurt performance in downstream tasks like retrieval or QA.

---

#### Comparison Table

| Feature                        | RecursiveCharacterTextSplitter | CharacterTextSplitter     |
| ------------------------------ | ------------------------------ | ------------------------- |
| Structure-aware splitting      |  Yes                          |  No                      |
| Preserves sentence/paragraphs  |  Yes                          |  No                      |
| Risk of splitting mid-sentence |  Minimal                     |  High                   |
| Ideal for RAG/document QA      |  Highly recommended           |  Only if structured text |
| Performance speed              |  Slightly slower             |  Faster                  |

---

#### Recommendation

Use `RecursiveCharacterTextSplitter` for most real-world document processing tasks, especially when building RAG pipelines or working with structured natural language content like PDFs or articles.

## Best Practices for Choosing Chunk Size in RAG

### Best Practices for Chunk Size in RAG

| Factor                      | Recommendation                                                                                                                                                                                          |
| ---------------------------| ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **LLM context limit**       | Choose a chunk size that lets you retrieve multiple chunks **without exceeding the model’s token limit**. For example, GPT-4o supports 128k tokens, but with GPT-3.5 (16k) or GPT-4 (32k), keep it modest. |
| **Chunk size (in characters)** | Typically: **500–1,000 characters** per chunk → ~75–200 tokens. This fits well for retrieval + prompt without context overflow.                                                                           |
| **Chunk size (in tokens)**  | If using token-based splitter (e.g. `TokenTextSplitter`): aim for **100–300 tokens** per chunk.                                                                                                            |
| **Chunk overlap**           | Use **overlap of 10–30%** (e.g., 100–300 characters or ~50 tokens) to preserve context across chunk boundaries and avoid cutting off important ideas mid-sentence.                                        |
| **Document structure**      | Use **`RecursiveCharacterTextSplitter`** to preserve semantic boundaries (paragraphs, sentences) instead of arbitrary cuts.                                                                                |
| **Task type**               | For **question answering**, smaller chunks (~500–800 chars) reduce noise.<br>For **summarization**, slightly larger chunks (~1000–1500) are OK.                                                          |
| **Embedding model**         | Some models (e.g., `text-embedding-3-large`) can handle long input. But still, smaller chunks give **finer-grained retrieval**, which improves relevance.                                                  |
| **Query type**              | If users ask **very specific questions**, small focused chunks are better. For broader queries, bigger chunks might help.                                                                                  |


### Rule of Thumb

| Use Case                 | Chunk Size      | Overlap |
| ------------------------| --------------- | ------- |
| Factual Q&A              | 500–800 chars   | 100–200 |
| Summarization            | 1000–1500 chars | 200–300 |
| Technical documents      | 400–700 chars   | 100–200 |
| Long reports/books       | 800–1200 chars  | 200–300 |
| Small LLMs (≤16k tokens) | ≤800 chars      | 100–200 |


### Avoid

- Chunks >2000 characters: risks context overflow.
- No overlap: may lose key information between chunks.



<h2 style="color: #FF8C00;">Embeddings</h2>

Embeddings transform text into dense vector representations, capturing semantic meaning and contextual relationships. They are essential for efficient document retrieval and similarity analysis.

- **What are OpenAI Embeddings?**
  - Pre-trained embeddings like `text-embedding-3-large` generate high-quality vector representations for text.
  - Encapsulate semantic relationships in the text, enabling robust NLP applications.

- **Key Features of `text-embedding-3-large`:**
  - Large-scale embedding model optimized for accuracy and versatility.
  - Handles diverse NLP tasks, including retrieval, classification, and clustering.
  - Ideal for applications with high-performance requirements.

- **Benefits:**
  - Reduces the need for extensive custom training.
  - Provides state-of-the-art performance in retrieval-augmented systems.
  - Compatible with RAGs to create powerful context-aware models.


In [19]:
%pip install langchain-openai
from langchain_openai import OpenAIEmbeddings

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv, find_dotenv

In [17]:
find_dotenv()

'c:\\Users\\nahia\\OneDrive\\Escritorio\\Ironhack_Bootcamp\\Woche_Sieben\\lab-intro-rag\\your-code\\.env'

In [18]:
load_dotenv()

True

In [19]:
api_key = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

<h2 style="color: #FF8C00;">ChromaDB</h2>

ChromaDB is a versatile vector database designed for efficiently storing and retrieving embeddings. It integrates seamlessly with embedding models to enable high-performance similarity search and context-based retrieval.

### Workflow Overview:
- **Step 1:** Generate embeddings using a pre-trained model (e.g., OpenAI's `text-embedding-3-large`).
- **Step 2:** Store the embeddings in ChromaDB for efficient retrieval and similarity calculations.
- **Step 3:** Use the stored embeddings to perform searches, matching, or context-based retrieval.

### Key Features of ChromaDB:
- **Scalability:** Handles large-scale datasets with optimized indexing and search capabilities.
- **Speed:** Provides fast and accurate retrieval of embeddings for real-time applications.
- **Integration:** Supports integration with popular frameworks and libraries for embedding generation.

In [28]:
pip install "langchain<0.1"

  Using cached langchain-0.0.354-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      + c:\Users\nahia\anaconda3\python.exe C:\Users\nahia\AppData\Local\Temp\pip-install-hcdpuae4\numpy_71322e8313354d168f87bc4da0ad0dd7\vendored-meson\meson\meson.py setup C:\Users\nahia\AppData\Local\Temp\pip-install-hcdpuae4\numpy_71322e8313354d168f87bc4da0ad0dd7 C:\Users\nahia\AppData\Local\Temp\pip-install-hcdpuae4\numpy_71322e8313354d168f87bc4da0ad0dd7\.mesonpy-xl306lmu -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\nahia\AppData\Local\Temp\pip-install-hcdpuae4\numpy_71322e8313354d168f87bc4da0ad0dd7\.mesonpy-xl306lmu\meson-python-native-file.ini
      The Meson build system
      Version: 1.2.99
      Source dir: C:\Users\nahia\AppData\Local\Temp\pip-install-hcdpuae4\numpy_71322e8313354d168f87bc4da0ad0dd7
      Build dir: C:\Users\nahia\AppData\Local\Temp\pip-install-hcdpuae4\numpy_71322e8313354d168f

In [29]:
import langchain
print(langchain.__version__)

1.0.8


In [30]:
!pip install langchain-community chromadb


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings 

In [21]:
db = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db_lesson")
print("ChromaDB created with document embeddings.")

ChromaDB created with document embeddings.


In [23]:
db = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db_LAB")
print("ChromaDB created with document embeddings.")

ChromaDB created with document embeddings.


<h1 style="color: #FF6347;">Retrieving Documents</h1>


### Exercice1: Write a user question that someone might ask about your book’s topic or content.

In [34]:
user_question = "how civil liberties can be compromised by AI?" # User question
retrieved_docs = db.similarity_search(user_question, k=10) # k is the number of documents to retrieve

In [38]:
# Display top results
for i, doc in enumerate(retrieved_docs[:3]): # Display top 3 results
    print(f"Document {i+1}:\n{doc.page_content[200:10000]}") # Display content

Document 1:
erate the concept of AI to alleviate some source of injustice and proceed to 
develop a technology that may be useful for a specific purpose, but does not 
have the backing of a significant entity to ensure that AI products are chosen to 
improve quality of life for the most vulnerable through a process of consulta -
tion and oversight.
Non-profits and other organisations dedicated to the alleviation of human 
suffering and improving justice are traditionally staffed by less technical per -
sons, such as those with legal training rather than technical training in software
Document 2:
lved are making sincere efforts 
to be fair (which they often aren’t) (Eubanks 2018). The data demands of AI 
mean that the pattern of having to trade private personal information for ser -
vices will become even more invasive. The optimisations of AI act as an inverse 
intersectionality, applying additional downward pressure on existing fissures in 
the social fabric. Like Eubanks, we should b

<h2 style="color: #FF8C00;">Preparing Content for GenAI</h2>

In [36]:
def _get_document_prompt(docs):
    prompt = "\n"
    for doc in docs:
        prompt += "\nContent:\n"
        prompt += doc.page_content + "\n\n"
    return prompt

In [37]:
# Generate a formatted context from the retrieved documents
formatted_context = _get_document_prompt(retrieved_docs)
print("Context formatted for GPT model.")

Context formatted for GPT model.


In [65]:
print(formatted_context[:500])



Content:
never satisfy rights-based or duty-based obligations to other humans such as 
protection from persistent surveillance. 
However, AI for social good is ad hoc. That is to say, private individuals gen-
erate the concept of AI to alleviate some source of injustice and proceed to 
develop a technology that may be useful for a specific purpose, but does not 
have the backing of a significant entity to ensure that AI products are chosen to 
improve quality of life for the most vulnerable th


<h2 style="color: #FF8C00;">ChatBot Architecture</h2>

### Exercice2: Write a prompt that is relevant and tailored to the content and style of your book.

In [74]:
prompt = f"""

## SYSTEM ROLE
You are a knowledgeable and factual chatbot designed to assist with technical questions about **AI Usage**, specifically focusing on **Critical Perspectives**.
Your answers must be based exclusively on provided content from technical books provided.

## USER QUESTION
The user has asked:
"{user_question}"

## CONTEXT
Here is the relevant content from the technical books:
'''
{formatted_context}
'''

## GUIDELINES
1. **Accuracy**:
   - Only use the content in the `CONTEXT` section to answer.
   - If the answer cannot be found, explicitly state: "The provided context does not contain this information."
   - Start explain AI  contribution to the society and civil liberties
   - Follow by current correlation between AI and civil liberties
   - Lastly explain AI legislation with civil liberties perspective.

2. **Transparency**:
   - Reference the book's name and page numbers when providing information.
   - Do not speculate or provide opinions.

3. **Clarity**:
   - Use simple, professional, and concise language.
   - Format your response in Markdown for readability.

## TASK
1. Answer the user's question **directly** if possible.
2. Point the user to relevant parts of the documentation.
3. Provide the response in the following format:

## RESPONSE FORMAT
'''
# [Brief Title of the Answer]
[Answer in simple, clear text.]

**Source**:
• [Book Title], Page(s): [...]
'''
"""
print("Prompt constructed.")

Prompt constructed.


In [59]:
import openai

### Exercice3: Tune parameters like temperature, and penalties to control how creative, focused, or varied the model's responses are.

In [60]:
# Set up GPT client and parameters
client = openai.OpenAI()
model_params = {
    'model': 'gpt-4o',
    'temperature': 0.7,  # Increase creativity
    'max_tokens':5000 ,  # Allow for longer responses
    'top_p': 0.9,        # Use nucleus sampling
    'frequency_penalty':0.4 ,  # Reduce repetition
    'presence_penalty': 0.6   # Encourage new topics
}

<h1 style="color: #FF6347;">Response</h1>


In [75]:
messages = [{'role': 'user', 'content': prompt}]
completion = client.chat.completions.create(messages=messages, **model_params, timeout=200)

In [76]:
print(prompt)



## SYSTEM ROLE
You are a knowledgeable and factual chatbot designed to assist with technical questions about **AI Usage**, specifically focusing on **Critical Perspectives**.
Your answers must be based exclusively on provided content from technical books provided.

## USER QUESTION
The user has asked:
"how civil liberties can be compromised by AI?"

## CONTEXT
Here is the relevant content from the technical books:
'''


Content:
never satisfy rights-based or duty-based obligations to other humans such as 
protection from persistent surveillance. 
However, AI for social good is ad hoc. That is to say, private individuals gen-
erate the concept of AI to alleviate some source of injustice and proceed to 
develop a technology that may be useful for a specific purpose, but does not 
have the backing of a significant entity to ensure that AI products are chosen to 
improve quality of life for the most vulnerable through a process of consulta -
tion and oversight.
Non-profits and other orga

In [55]:
answer = completion.choices[0].message.content
print(answer)

To provide a comprehensive answer on how civil liberties can be compromised by AI, I will focus on several key aspects often discussed in critical perspectives of AI usage:

1. **Surveillance**: AI technologies, especially those used in surveillance systems, can significantly compromise civil liberties. Facial recognition and other monitoring tools can lead to mass surveillance, where individuals are constantly monitored without their consent. This can infringe on the right to privacy and lead to a chilling effect on free speech and assembly as people may alter their behavior due to the awareness of being watched.

2. **Bias and Discrimination**: AI systems often reflect and perpetuate existing biases found in the data they are trained on. This can result in discriminatory outcomes that affect people's civil rights, such as biased hiring algorithms or unfair law enforcement practices that disproportionately target minority communities. Such discrimination undermines equality before the

<img src="https://miro.medium.com/v2/resize:fit:824/1*GK56xmDIWtNQAD_jnBIt2g.png" alt="NLP Gif" style="width: 500px">

<h2 style="color: #FF6347;">Cosine Similarity</h2>

**Cosine similarity** is a metric used to measure the alignment or similarity between two vectors, calculated as the cosine of the angle between them. It is the **most common metric used in RAG pipelines** for vector retrieval.. It provides a scale from -1 to 1:

- **-1**: Vectors are completely opposite.
- **0**: Vectors are orthogonal (uncorrelated or unrelated).
- **1**: Vectors are identical.


<img src="https://storage.googleapis.com/lds-media/images/cosine-similarity-vectors.original.jpg" alt="NLP Gif" style="width: 700px">

<h2 style="color: #FF6347;">Keyword Highlighting</h2>

Highlighting important keywords helps users quickly understand the relevance of the retrieved text to their query.

In [56]:
from termcolor import colored

The `highlight_keywords` function is designed to highlight specific keywords within a given text. It replaces each keyword in the text with a highlighted version using the `colored` function from the `termcolor` library.


In [63]:
def highlight_keywords(text, keywords):
    for keyword in keywords:
        text = text.replace(keyword, colored(keyword, 'green', attrs=['bold']))
    return text

### Exercice4: add your keywords

In [73]:
query_keywords = ["civil", "liberties", "human", "rights" ] # add your keywords
for i, doc in enumerate(retrieved_docs[:1]):
    snippet = doc.page_content[:200]
    highlighted = highlight_keywords(snippet, query_keywords)
    print(f"Snippet {i+1}:\n{highlighted}\n{'-'*80}")

Snippet 1:
never satisfy rights-based or duty-based obligations to other humans such as 
protection from persistent surveillance. 
However, AI for social good is ad hoc. That is to say, private individuals gen-

--------------------------------------------------------------------------------


1. `query_keywords` is a list of keywords to be highlighted.
2. The loop iterates over the first document in retrieved_docs.
3. For each document, a snippet of the first 200 characters is extracted.
4. The highlight_keywords function is called to highlight the keywords in the snippet.
5. The highlighted snippet is printed along with a separator line.

<h1 style="color: #FF6347;">Bonus</h1>

**Try loading one of your own PDF books and go through the steps again to explore how the pipeline works with your content**:
